## Import libraires needed
Import the following libraries:

In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

##  1. Installing and configuring CPDCTL <a class="anchor" id="part1"></a>

Download the binary and then display the version number:

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']
        
response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.132


### 2  Adding CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

In [5]:
CPD_USER_NAME = 'dhshi'
CPD_USER_PASSWORD = 'mypassword'
CPD_URL = 'https://cpd-cpd-instance.apps.cp4d404ugi.cp.fyre.ibm.com'

Add "cpd_user" user to the cpdctl configuration:

In [6]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd" cluster to the cpdctl configuration:

In [7]:
! cpdctl config profile set cpd --url {CPD_URL}

Add "cpd" context to the cpdctl configuration:

In [8]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts:

In [9]:
! cpdctl config context list

Name                          Profile                       User                       Current   
inClusterEnvironmentContext   inClusterEnvironmentProfile   inClusterEnvironmentUser   *   


Switch to the context you just created if it is not marked in the `Current` column:

In [10]:
! cpdctl config context use inClusterEnvironmentContext

Switched to context "inClusterEnvironmentContext".


List available spaces in context:

In [12]:
! cpdctl space list

...
ID                                     Name                                                 Created                    Description   State    Tags   
f1c213be-597f-4c32-bcd5-9e4344ead75c   AutoAI-TD-Sub-Deployment-Space                       2022-01-25T01:59:19.769Z                 active   []   
83de1cda-6129-4f97-a464-b71ec5224393   r-shiny-test-space                                   2022-03-23T22:04:12.062Z                 active   []   
ef5ae6e4-17be-4e46-af04-6a8f6ace3eae   julian-test                                          2022-03-29T09:00:24.454Z                 active   []   
cd4e2877-1afc-4d02-a2cb-386020912e44   openscale-express-path-00000000-0000-0000-0000-16…   2022-03-30T15:10:33.888Z                 active   []   
2e1d6881-c039-4894-8fa2-220a01c9d4af   openscale-express-path-preprod-00000000-0000-0000…   2022-03-30T15:10:51.098Z                 active   []   
e5a47063-9735-48fc-bd1a-45ed88ebf425   cpdctl-samples-project-notebook-space-test           2022-04-03T21: